In [ ]:
import pickle
import gzip
import numpy as np
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import train_test_split 
from sklearn import metrics 
# import bentoml

### Data Loading

In [ ]:
f = gzip.open('../data/mnist.pkl.gz', 'rb')
training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
f.close()

In [ ]:
fig = plt.figure()
for i in range(6):
    t = training_data[0][i]
    t = t.reshape(-1, 28)
    
    
    plt.subplot(2, 3, i+1)
    plt.tight_layout()
    plt.imshow(t, cmap="gray", interpolation='none')
    plt.title(f"Ground Truth: {training_data[1][i]}")
    plt.xticks([])
    plt.yticks([])

### SVM for Baseline on MNIST Data

In [ ]:
clf = svm.SVC()
clf.fit(training_data[0], training_data[1])

In [ ]:
preds = [int(a) for a in clf.predict(test_data[0])]
num_correct = sum(int(a==y) for a, y in zip(preds, test_data[1]))
print("Baseline classifier using an SVM.")
print(str(num_correct) + " of " + str(len(test_data[1])) + " values correct.")
print(f"Accuracy: {num_correct/len(test_data[1])}")

In [ ]:
clf = DecisionTreeClassifier(criterion="entropy", max_depth=100)
clf.fit(training_data[0], training_data[1])

In [ ]:
preds = [int(a) for a in clf.predict(test_data[0])]
num_correct = sum(int(a==y) for a, y in zip(preds, test_data[1]))
print("Baseline classifier using a Decision Tree.")
print(str(num_correct) + " of " + str(len(test_data[1])) + " values correct.")
print(f"Accuracy: {num_correct/len(test_data[1])}")

### Network

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import tensor
from torch.utils.data import TensorDataset, DataLoader
from torchvision.datasets import MNIST
from torchvision.transforms import Compose, ToTensor, Normalize

In [ ]:
# train_loader = DataLoader(
#   MNIST('./files/', train=True, download=True,
#                              transform=Compose([
#                                ToTensor(),
#                                Normalize(
#                                  (0.1307,), (0.3081,))
#                              ])),
#   batch_size=batch_size_train, shuffle=True)

# test_loader = DataLoader(
#   MNIST('./files/', train=False, download=True,
#                              transform=Compose([
#                                ToTensor(),
#                                Normalize(
#                                  (0.1307,), (0.3081,))
#                              ])),
#   batch_size=batch_size_test, shuffle=True)

In [ ]:
# print(len(train_loader))
# print(len(train_loader.dataset))

#### Model

In [ ]:
n_epochs = 40
batch_size_train = 64
batch_size_test = 1000
learning_rate = 0.008
momentum = 0.5
log_interval = 100

random_seed = 1
torch.backends.cudnn.enabled = False
torch.manual_seed(random_seed)

In [ ]:
f = gzip.open('mnist/DeepLearningPython/mnist.pkl.gz', 'rb')
training_data, validation_data, test_data = pickle.load(f, encoding="latin1")
f.close()

In [ ]:
train_2d = [im.reshape(-1, 28) for im in training_data[0]]
test_2d = [im.reshape(-1, 28) for im in test_data[0]]
valid_2d = [im.reshape(-1, 28) for im in validation_data[0]]

training_data = TensorDataset(tensor(train_2d), tensor(training_data[1]))
train_loader = DataLoader(training_data, batch_size=batch_size_train, shuffle=True)
test_data = TensorDataset(tensor(test_2d), tensor(test_data[1]))
test_loader = DataLoader(test_data, batch_size=batch_size_test, shuffle=True)
# validation_data = TensorDataset(tensor(valid_2d), tensor(validation_data[1]))
# validation_loader = DataLoader(validation_data)

In [ ]:
len(train_2d)

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        # x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2(x), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)


In [ ]:
network = Network()
optimizer = optim.SGD(network.parameters(), lr=learning_rate)

#### Training

In [ ]:
train_losses = []
train_counter = []
test_losses = []
test_counter = [i*len(train_loader.dataset) for i in range(n_epochs+1)]

def train(epoch):
    network.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.reshape(-1, 1, 28, 28)
        optimizer.zero_grad()
        output = network(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), len(train_loader.dataset),100. * batch_idx / len(train_loader), loss.item()))
            train_losses.append(loss.item())
            train_counter.append((batch_idx*64) + ((epoch-1)*len(train_loader.dataset)))
            torch.save(network.state_dict(), './results/model.pth')
            torch.save(optimizer.state_dict(), './results/optimizer.pth')
            

In [ ]:
def test():
    network.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data = data.reshape(-1, 1, 28, 28)
            output = network(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).sum()
    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print(f'\nTest set: Avg. loss: {test_loss}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset)}%)\n')

In [ ]:
test()
for epoch in range(1, n_epochs + 1):
    train(epoch)
    test()

In [ ]:
fig = plt.figure()
plt.plot(train_counter, train_losses, color='blue')
plt.scatter(test_counter, test_losses, color='red')
plt.legend(['Train Loss', 'Test Loss'], loc='upper right')
plt.xlabel('number of training examples seen')
plt.ylabel('negative log likelihood loss')

In [ ]:
examples = enumerate(test_loader)
batch_idx, (example_data, example_targets) = next(examples)
ex = example_data.reshape(-1, 1, 28, 28)

In [ ]:
with torch.no_grad():
    
    output = network(ex)

In [ ]:
ex[0].shape

In [ ]:
output = network(ex[0])
output

In [ ]:
type(output)

In [ ]:
output.data.max(1, keepdim=True)[1].item()

In [ ]:
example_targets[0]

In [ ]:
fig = plt.figure(figsize=(14, 6))
for i in range(10):
    plt.subplot(2,5,i+1)
    #plt.tight_layout()
    plt.imshow(example_data[i], cmap='gray', interpolation='none')
    plt.title("Prediction: {} Truth: {}".format(output.data.max(1, keepdim=True)[1][i].item(), example_targets[i]))
    plt.xticks([])
    plt.yticks([])